In [1]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [2]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [3]:
df = pd.read_sql_query("""SELECT * FROM Facilities as f INNER JOIN Bookings as b ON f.facid = b.facid 
                            INNER JOIN Members as m ON b.memid = m.memid;""",engine)

In [4]:
df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,bookid,facid,memid,starttime,slots,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,3,Table Tennis,0.0,5.0,320,10,0,3,1,2012-07-03 11:00:00,2,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
1,4,Massage Room 1,9.9,80.0,4000,3000,1,4,1,2012-07-03 08:00:00,2,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,6,Squash Court,3.5,17.5,5000,80,2,6,0,2012-07-03 18:00:00,2,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
3,7,Snooker Table,0.0,5.0,450,15,3,7,1,2012-07-03 19:00:00,2,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
4,8,Pool Table,0.0,5.0,400,15,4,8,1,2012-07-03 10:00:00,1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05


In [5]:
df['revenue'] = (df[df['firstname'] == 'GUEST']['slots']*df['guestcost']).fillna(0) \
                + (df[df['firstname'] != 'GUEST']['slots']*df['membercost']).fillna(0)

In [6]:
fac_list = df[['name', 'revenue']].groupby('name').sum()
fac_list = fac_list[fac_list['revenue']<1000]
fac_list

,revenue
name,
Pool Table,270.0
Snooker Table,240.0
Table Tennis,180.0


## Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [7]:
df1 = pd.read_sql_query("""SELECT m1.memid AS ID, m1.surname || ', ' || m1.firstname AS member_name, 
                            m2.surname || ', ' || m2.firstname AS recommender FROM Members as m1
                            INNER JOIN Members as m2 ON m1.recommendedby = m2.memid 
                            WHERE m1.firstname != 'GUEST' AND m2.firstname != 'GUEST' 
                            GROUP BY member_name 
                            ORDER BY member_name""", engine)
df1.set_index('ID')

,member_name,recommender
ID,,
15,"Bader, Florence","Stibbons, Ponder"
12,"Baker, Anne","Stibbons, Ponder"
16,"Baker, Timothy","Farrell, Jemima"
8,"Boothe, Tim","Rownam, Tim"
5,"Butters, Gerald","Smith, Darren"
22,"Coplin, Joan","Baker, Timothy"
36,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
20,"Genting, Matthew","Butters, Gerald"


## Q12: Find the facilities with their usage by member, but not guests

In [8]:
df2 = pd.read_sql_query("""SELECT firstname || ' ' || surname AS member_name, name AS facility, bookid, slots
                            FROM Bookings AS b
                            INNER JOIN Facilities as f
                            ON b.facid = f.facid
                            INNER JOIN Members as m ON b.memid = m.memid
                            WHERE m.memid != 0""", engine)

In [9]:
df2['booking_count'] = 1
df2['total_slots'] = df2['slots']*df2['booking_count']
fac_usage = df2.drop(['bookid','slots'], axis = 1)

In [10]:
fac_usage.groupby(['member_name','facility']).sum()

booking_count  total_slots
member_name    facility                                   
Anna Mackenzie Badminton Court             30           96
               Massage Room 1               1            2
               Pool Table                  70           83
               Snooker Table                7           14
               Squash Court                 2            4
...                                       ...          ...
Tracy Smith    Snooker Table               45           90
               Squash Court                 6           12
               Table Tennis                28           56
               Tennis Court 1              30           93
               Tennis Court 2               2            6

[202 rows x 2 columns]

## Q13: Find the facilities usage by month, but not guests

In [11]:
df3 = pd.read_sql_query("""SELECT STRFTIME('%m', starttime) AS month, count(bookid) AS booking_count, 
                            SUM(slots) AS total_slots
                            FROM Bookings as b
                            INNER JOIN Facilities as f ON b.facid = f.facid
                            WHERE memid != 0
                            GROUP BY month""", engine)
df3

,month,booking_count,total_slots
0,07,480,1061
1,08,1168,2531
2,09,1512,3199
